In [37]:
import pandas as pd

# Load your match data
df = pd.read_csv("match_history.csv")

# Choose your character
my_name = "Abigail"

# Ensure numeric fields are properly typed
numeric_cols = ["Kills", "Assists", "Deaths", "Dmg_Player", "GameID", "Place"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Step 1: Get all matches where you played
my_matches = df[df["CharacterName"] == my_name]
my_game_ids = my_matches["GameID"].unique()

# Step 2: For each game, find teammates based on GameID + Place
teamups = []

for game_id in my_game_ids:
    game_data = df[df["GameID"] == game_id]
    
    my_row = game_data[game_data["CharacterName"] == my_name]
    if my_row.empty:
        continue
    
    my_place = my_row.iloc[0]["Place"]

    # Same match, same team, not me
    teammates = game_data[
        (game_data["Place"] == my_place) &
        (game_data["CharacterName"] != my_name)
    ].copy()

    if not teammates.empty:
        teammates["GameID"] = game_id  # Keep track of match
        teamups.append(teammates)

# Step 3: Combine and analyze teammates
if not teamups:
    print("⚠️ No teammates found using GameID + Place logic.")
else:
    df_teamups = pd.concat(teamups)

    # Group by teammate and calculate average stats
    teammate_stats = df_teamups.groupby("CharacterName").agg({
        "Kills": "mean",
        "Assists": "mean",
        "Deaths": "mean",
        "Dmg_Player": "mean",
    })

    # Sort: most games together, then damage
    teammate_stats = teammate_stats.sort_values(
        by=["Dmg_Player"], ascending=[False]
    )

    # Output
    if teammate_stats.empty:
        print("⚠️ No teammates meet the minimum match threshold.")
    else:
        print(f"\n🤖 ML Recommendation: Top 4 Teammates for {my_name}")
        print(teammate_stats.head(4))




🤖 ML Recommendation: Top 4 Teammates for Abigail
               Kills  Assists  Deaths  Dmg_Player
CharacterName                                    
Bernice         11.0      5.0     2.0     32095.0
Katja           11.0      4.0     4.0     28748.0
Vanya            2.0     12.0     2.0     23776.0
Eleven           3.0     16.0     1.0     10007.0
